## Módulos

In [110]:

#Módulo de Caixa preta --> Algoritmo Simplex


from mip import *
from collections import deque


# Módulo para criar o modelo e resolver o problema de programação linear

def save(model, filename):

    # Função para salvar o modelo em um arquivo .lp e imprimir na tela

    model.write(filename)
    with open(filename, "r") as f:
        print(f.read())



def solve(model):

    # Função para resolver o modelo e imprimir na tela

    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value:.2f}\n")

    print("Solution:")
    for v in model.vars:
        print(f"{v.name} = {v.x:.2f}")


def CreateModel(QtdVar,QtdRes,z,restricoes):

    # Função para criar o modelo usando dados retirados da instância
    # Recebe strings da função
    # Retorna o modelo criado
    print("Criando modelo...")
    print(f"Qtd de Variaveis: {QtdVar}, Qtd de Restricoes: {QtdRes}")


    model = Model(sense=MAXIMIZE, solver_name=CBC)
    x = {i: model.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in range(QtdVar)}

    model.objective = xsum(z[i]*x[i] for i in range(QtdVar)) # Max(z1*x1 + z2*x2 + ... + zn*xn))

    # S.A:
    for j in range(QtdRes):
       model += xsum(restricoes[j][i]*x[i] for i in range(QtdVar)) <= restricoes[j][QtdVar]



    save(model, "modelo.lp")
    solve(model)
    return model





def info(model):

    print("\nInformações do modelo:")
    print("Z = ", model.objective_value)
    print("Valores das variáveis:\n")
    print("x = ", [x.x for x in model.vars])
    print("\n")



In [111]:
#Modulos para leitura de arquivos

def read_file(file_name):

    #Funçao destinada a ler arquivo das instancias
    #recebe como parametro o nome do arquivo
    #retorna a quantidade de variaveis, quantidade de restriçoes, vetor de custos e matriz de restriçoes

    file = open(file_name, 'r')
    lines = file.readlines()
    file.close()

    # Separa as informações e converte para int
    QtdVar,QtdRes = [int(i) for i in lines[0].strip().split(' ')]
    z = [int(i) for i in lines[1].strip().split(' ')]
    restricoes = [[int(j) for j in i.strip().split(' ')] for i in lines[2:]]


    return QtdVar,QtdRes,z,restricoes





In [185]:
def Bound(model, limitante):
    # Função para restringir as ramificações do modelo
    # Retorna True se o modelo deve ser podado, False caso contrário
    
    

    if all(i.x is not None and i.x.is_integer() for i in model.vars):
        print(f"RAMO Z = {model.objective_value} -- PODADO")
        print("Poda por integralidade\n")
        info(model)
        return True

    elif model.num_solutions == 0 :
        print(f"RAMO Z = {model.objective_value} -- PODADO")
        print("Poda por inviabilidade\n")
        return True



    elif model.objective_bound < limitante:
        print(f"RAMO Z = {model.objective_value} -- PODADO")
        print("Poda por limitante\n")
        return True

    return False

def Branch(model):
    # Função para ramificar o modelo
    # Retorna dois novos modelos, um para xi = 0 e outro para xi = 1

    minimum = 0.5
    resto = 0
    varIndex = 0

    for i in range(len(model.vars)):
        resto = model.vars[i].x % 1
        resto = abs(model.vars[i].x % 1 - 0.5)
        if resto < minimum:
            minimum = resto
            varIndex = i

    print(f"\nVariável escolhida para ramificar: {varIndex}")
    print(f"Valor fracionário: {model.vars[varIndex].x}")
    print(f"Valor fracionário mais próximo de 0.5: {minimum}")
    print(f"\nZ = {model.objective_value}")

    modelEsquerda = model.copy()
    modelEsquerda += modelEsquerda.vars[varIndex] == 0

    modelDireita = model.copy()
    modelDireita += modelDireita.vars[varIndex] == 1

    modelEsquerda.optimize()
    modelDireita.optimize()

    return modelEsquerda, modelDireita




# Define a função de busca em largura
def breadth_first_search(initial_model, limitante):
    queue = deque([initial_model])
    
    
    penult_model = None  # Variável para armazenar o penúltimo modelo
    last_model = None  # Variável para armazenar o último modelo
    model = None

    while queue:
        
        print(f"penult_model = {penult_model.objective_value if penult_model is not None else None}")
        if last_model != None and last_model.objective_value != None:
            penult_model = last_model
        last_model = model
        model = queue.popleft()  # Retira o modelo da frente da fila
        
        # Verifica se o modelo deve ser podado
        if not Bound(model, limitante):
            left_model, right_model = Branch(model)

            # Verifica se os modelos à esquerda e à direita não são nulos e têm variáveis
            if left_model is not None and len(left_model.vars) > 0:
                queue.append(left_model)
            if right_model is not None and len(right_model.vars) > 0:
                queue.append(right_model)



    print("\nSAIDAS:\n")
    print(f"penult_model = {penult_model.objective_value if penult_model is not None else None}")
    print(f"last_model = {last_model.objective_value if last_model is not None else None}")
    # Verifica se a fila está vazia (último modelo foi pop)
    if last_model is not None:
        # A última coisa que foi processada é armazenada em last_model
        
        if penult_model.objective_value > last_model.objective_value:
            print("\nSolução Ótima:\n")
            info(penult_model)
        else:
        
            print("\nSolução Ótima:\n")
            info(last_model)


## Teste

#### Caso 0

In [188]:
r = read_file("I0.txt")
m = CreateModel(r[0],r[1],r[2],r[3])

Criando modelo...
Qtd de Variaveis: 3, Qtd de Restricoes: 2
\Problem name: 

Minimize
OBJROW: -5 x_0 - x_1
Subject To
constr(0):  3 x_0 + 5 x_1 + 2 x_2 <= 6
constr(1):  4 x_0 + 4 x_1 + 4 x_2 <= 7
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solution value  = 8.75

Solution:
x_0 = 1.75
x_1 = 0.00
x_2 = 0.00


In [189]:
limitante =  float("-inf")# Inicializa o limitante como infinito ou define um valor apropriado
breadth_first_search(m, limitante)

penult_model = None

Variável escolhida para ramificar: 0
Valor fracionário: 1.75
Valor fracionário mais próximo de 0.5: 0.25

Z = 8.75
penult_model = None

Variável escolhida para ramificar: 1
Valor fracionário: 1.2
Valor fracionário mais próximo de 0.5: 0.30000000000000004

Z = 1.2
penult_model = None

Variável escolhida para ramificar: 1
Valor fracionário: 0.6
Valor fracionário mais próximo de 0.5: 0.09999999999999998

Z = 5.6
penult_model = 8.75
RAMO Z = 0.0 -- PODADO
Poda por integralidade


Informações do modelo:
Z =  0.0
Valores das variáveis:

x =  [0.0, 0.0, 0.0]


penult_model = 1.2
RAMO Z = 1.0 -- PODADO
Poda por integralidade


Informações do modelo:
Z =  1.0
Valores das variáveis:

x =  [0.0, 1.0, 0.0]


penult_model = 5.6
RAMO Z = 5.0 -- PODADO
Poda por integralidade


Informações do modelo:
Z =  5.0
Valores das variáveis:

x =  [1.0, 0.0, 0.0]


penult_model = 0.0
RAMO Z = None -- PODADO
Poda por inviabilidade


SAIDAS:

penult_model = 1.0
last_model = 5.0

Solução Ótima

#### Caso 1

In [190]:
r = read_file("I1.txt")
m = CreateModel(r[0],r[1],r[2],r[3])

Criando modelo...
Qtd de Variaveis: 7, Qtd de Restricoes: 11
\Problem name: 

Minimize
OBJROW: -2 x_0 -10 x_1 -8 x_2 -7 x_3 -10 x_4 -10 x_5 -6 x_6
Subject To
constr(0):  5 x_0 + 7 x_1 + 8 x_2 + x_3 + 7 x_4 + 5 x_5 + 6 x_6 <= 20
constr(1):  x_0 + 6 x_1 + 4 x_2 + 9 x_3 + 10 x_4 + 6 x_5 + 10 x_6 <= 30
constr(2):  4 x_0 + 4 x_1 + 4 x_2 + x_3 + 5 x_4 + 5 x_5 + 10 x_6 <= 40
constr(3):  3 x_0 + 10 x_1 + 8 x_2 + x_3 + 3 x_4 + 3 x_5 + 8 x_6 <= 30
constr(4):  10 x_0 + 8 x_1 + 9 x_2 + 9 x_3 + 7 x_4 + 6 x_5 + 10 x_6 <= 20
constr(5):  6 x_0 + 6 x_1 + 3 x_2 + 6 x_3 + 3 x_4 + 7 x_5 + 2 x_6 <= 80
constr(6):  7 x_0 + 10 x_1 + 7 x_2 + 8 x_3 + 7 x_4 + 8 x_5 + 7 x_6 <= 100
constr(7):  9 x_0 + 8 x_1 + x_2 + x_3 + 8 x_4 + 10 x_5 + 2 x_6 <= 90
constr(8):  x_0 + 5 x_1 + 3 x_2 + 10 x_3 + 2 x_4 + 4 x_5 + 9 x_6 <= 70
constr(9):  9 x_0 + 6 x_1 + x_2 + 4 x_3 + 7 x_4 + 5 x_5 + 10 x_6 <= 60
constr(10):  5 x_0 + 7 x_1 + 4 x_2 + 4 x_3 + 3 x_4 + 4 x_5 + 10 x_6 <= 40
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solu

In [191]:
limitante =  float("-inf")# Inicializa o limitante como infinito ou define um valor apropriado
breadth_first_search(m, limitante)

penult_model = None

Variável escolhida para ramificar: 5
Valor fracionário: 3.333333333333333
Valor fracionário mais próximo de 0.5: 0.16666666666666696

Z = 33.33333333333333
penult_model = None

Variável escolhida para ramificar: 4
Valor fracionário: 2.8571428571428568
Valor fracionário mais próximo de 0.5: 0.35714285714285676

Z = 28.57142857142857
penult_model = None
RAMO Z = 30.0 -- PODADO
Poda por integralidade


Informações do modelo:
Z =  30.0
Valores das variáveis:

x =  [0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0]


penult_model = 33.33333333333333

Variável escolhida para ramificar: 1
Valor fracionário: 2.5
Valor fracionário mais próximo de 0.5: 0.0

Z = 25.0
penult_model = 28.57142857142857

Variável escolhida para ramificar: 1
Valor fracionário: 1.625
Valor fracionário mais próximo de 0.5: 0.125

Z = 26.25
penult_model = 30.0

Variável escolhida para ramificar: 2
Valor fracionário: 2.2222222222222223
Valor fracionário mais próximo de 0.5: 0.2777777777777777

Z = 17.77777777777778


#### Caso 2

In [192]:
r = read_file("I2.txt")
m = CreateModel(r[0],r[1],r[2],r[3])

Criando modelo...
Qtd de Variaveis: 9, Qtd de Restricoes: 9
\Problem name: 

Minimize
OBJROW: -7 x_0 -7 x_1 -7 x_2 -5 x_3 -8 x_4 -8 x_5 -9 x_6 -10 x_7 -7 x_8
Subject To
constr(0):  x_0 + 3 x_1 + x_2 + 3 x_3 + 3 x_4 + 7 x_5 + 2 x_6 + x_7 + 4 x_8 <= 80
constr(1):  7 x_0 + 6 x_1 + 10 x_2 + x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 7 x_7 + x_8 <= 90
constr(2):  3 x_0 + 2 x_1 + x_2 + 3 x_3 + 3 x_4 + 2 x_5 + x_6 + 6 x_7 + 5 x_8 <= 10
constr(3):  10 x_0 + 8 x_1 + 3 x_2 + 6 x_3 + 10 x_4 + 7 x_5 + 3 x_6 + 10 x_7 + 4 x_8 <= 30
constr(4):  2 x_0 + 8 x_1 + 6 x_2 + 5 x_3 + 6 x_4 + 6 x_5 + 9 x_6 + 7 x_7 + 2 x_8 <= 80
constr(5):  3 x_0 + 10 x_1 + x_2 + 9 x_3 + 2 x_4 + 7 x_5 + 7 x_6 + 9 x_7 + 10 x_8 <= 90
constr(6):  x_0 + 7 x_1 + 10 x_2 + 10 x_3 + 5 x_4 + 2 x_5 + 9 x_6 + 10 x_7 + 2 x_8 <= 20
constr(7):  10 x_0 + 3 x_1 + 2 x_2 + 3 x_3 + 10 x_4 + 2 x_5 + x_6 + 9 x_7 + 7 x_8 <= 10
constr(8):  2 x_0 + x_1 + 7 x_2 + 10 x_3 + x_4 + x_5 + 2 x_6 + x_7 + x_8 <= 50
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solution

In [193]:
limitante =  float("-inf")# Inicializa o limitante como infinito ou define um valor apropriado
breadth_first_search(m, limitante)

penult_model = None

Variável escolhida para ramificar: 5
Valor fracionário: 3.573573573573572
Valor fracionário mais próximo de 0.5: 0.07357357357357186

Z = 42.49249249249248
penult_model = None

Variável escolhida para ramificar: 8
Valor fracionário: 1.1475409836065573
Valor fracionário mais próximo de 0.5: 0.3524590163934427

Z = 25.737704918032787
penult_model = None

Variável escolhida para ramificar: 6
Valor fracionário: 1.80327868852459
Valor fracionário mais próximo de 0.5: 0.30327868852458995

Z = 30.42622950819672
penult_model = 42.49249249249248

Variável escolhida para ramificar: 0
Valor fracionário: 0.7865168539325843
Valor fracionário mais próximo de 0.5: 0.2865168539325843

Z = 24.719101123595507
penult_model = 25.737704918032787

Variável escolhida para ramificar: 0
Valor fracionário: 0.10112359550561802
Valor fracionário mais próximo de 0.5: 0.398876404494382

Z = 25.606741573033705
penult_model = 30.42622950819672

Variável escolhida para ramificar: 1
Valor fracionár

#### Caso 3

In [194]:
r = read_file("I3.txt")
m = CreateModel(r[0],r[1],r[2],r[3])

Criando modelo...
Qtd de Variaveis: 9, Qtd de Restricoes: 12
\Problem name: 

Minimize
OBJROW: -7 x_0 -9 x_1 -10 x_2 -3 x_3 -6 x_4 - x_5 -9 x_6 -8 x_7 -8 x_8
Subject To
constr(0):  2 x_0 + x_1 + 9 x_2 + 6 x_3 + 3 x_4 + 6 x_5 + 10 x_6 + 9 x_7 + x_8 <= 60
constr(1):  8 x_0 + 6 x_1 + 6 x_2 + 5 x_3 + 2 x_4 + 2 x_5 + 4 x_6 + 3 x_7 + 6 x_8 <= 80
constr(2):  8 x_0 + x_1 + 3 x_2 + 7 x_3 + x_4 + 4 x_5 + 8 x_6 + 3 x_7 + 4 x_8 <= 30
constr(3):  6 x_0 + 3 x_1 + 9 x_2 + 5 x_3 + 9 x_4 + 6 x_5 + 9 x_6 + 9 x_7 + 6 x_8 <= 40
constr(4):  10 x_0 + 8 x_1 + 8 x_2 + 7 x_3 + 10 x_4 + 10 x_5 + 9 x_6 + 9 x_7 + 3 x_8 <= 20
constr(5):  10 x_0 + 10 x_1 + 10 x_2 + 9 x_3 + 10 x_4 + x_5 + 8 x_6 + 3 x_7 + 10 x_8 <= 90
constr(6):  10 x_0 + 5 x_1 + 8 x_2 + 2 x_3 + 7 x_4 + 8 x_5 + 6 x_6 + 2 x_7 + 2 x_8 <= 90
constr(7):  7 x_0 + 9 x_1 + x_2 + 9 x_3 + 5 x_4 + 8 x_5 + 5 x_6 + 9 x_7 + 5 x_8 <= 80
constr(8):  2 x_0 + 4 x_1 + 6 x_2 + x_3 + 7 x_4 + 9 x_5 + 10 x_6 + x_7 + 7 x_8 <= 80
constr(9):  8 x_0 + 10 x_1 + 2 x_2 + 6 x_3 +

In [195]:
limitante = float("-inf") #Inicializa o limitante como infinito ou define um valor apropriado
breadth_first_search(m, limitante)

penult_model = None

Variável escolhida para ramificar: 8
Valor fracionário: 0.6060606060606062
Valor fracionário mais próximo de 0.5: 0.10606060606060619

Z = 27.575757575757578
penult_model = None

Variável escolhida para ramificar: 2
Valor fracionário: 2.5
Valor fracionário mais próximo de 0.5: 0.0

Z = 25.0
penult_model = None

Variável escolhida para ramificar: 2
Valor fracionário: 0.5
Valor fracionário mais próximo de 0.5: 0.0

Z = 13.0
penult_model = 27.575757575757578

Variável escolhida para ramificar: 6
Valor fracionário: 1.6216216216216217
Valor fracionário mais próximo de 0.5: 0.12162162162162171

Z = 20.675675675675677
penult_model = 25.0

Variável escolhida para ramificar: 1
Valor fracionário: 0.6486486486486487
Valor fracionário mais próximo de 0.5: 0.14864864864864868

Z = 22.648648648648653
penult_model = 13.0

Variável escolhida para ramificar: 6
Valor fracionário: 0.5
Valor fracionário mais próximo de 0.5: 0.0

Z = 12.5
penult_model = 20.675675675675677
RAMO Z = None

#### Caso 4

In [196]:
r = read_file("I4.txt")
m = CreateModel(r[0],r[1],r[2],r[3])

Criando modelo...
Qtd de Variaveis: 9, Qtd de Restricoes: 9
\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2 + 6 x_3 + 10 x_4 + 8 x_5 + x_6 + 4 x_7 + 4 x_8 <= 70
constr(7):  9 x_0 + x_1 + 9 x_2 + 7 x_3 + 10 x_4 + 5 x_5 + 6 x_6 + 2 x_7 + 5 x_8 <= 10
constr(8):  2 x_0 + 7 x_1 + 6 x_2 + 5 x_3 + x_4 + x_5 + 9 x_6 + 2 x_7 + x_8 <= 20
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solution value

In [197]:
limitante =  float("-inf")# Inicializa o limitante como infinito ou define um valor apropriado
breadth_first_search(m, limitante)

penult_model = None

Variável escolhida para ramificar: 2
Valor fracionário: 0.7142857142857143
Valor fracionário mais próximo de 0.5: 0.2142857142857143

Z = 13.571428571428573
penult_model = None

Variável escolhida para ramificar: 3
Valor fracionário: 0.3846153846153847
Valor fracionário mais próximo de 0.5: 0.11538461538461531

Z = 13.076923076923077
penult_model = None

Variável escolhida para ramificar: 1
Valor fracionário: 0.3243243243243243
Valor fracionário mais próximo de 0.5: 0.1756756756756757

Z = 13.486486486486486
penult_model = 13.571428571428573

Variável escolhida para ramificar: 1
Valor fracionário: 0.37634408602150554
Valor fracionário mais próximo de 0.5: 0.12365591397849446

Z = 12.688172043010752
penult_model = 13.076923076923077

Variável escolhida para ramificar: 8
Valor fracionário: 0.5945945945945947
Valor fracionário mais próximo de 0.5: 0.09459459459459474

Z = 12.54054054054054
penult_model = 13.486486486486486

Variável escolhida para ramificar: 7
Valor f